# Data Preparation & Filtering

In [54]:
# import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings("ignore")

In [55]:
# Load main dataset
train_df = pd.read_csv('data/train_processed.csv')
train_df['date'] = pd.to_datetime(train_df['date'])

In [56]:
train_df.head()

,Unnamed: 0,store_nbr,item_nbr,date,id,unit_sales,onpromotion,year,month,day,day_of_week,unit_sales_7d_avg
0,0,24,105574,2013-01-02,18790.0,12.0,False,2013,1,2,2,NaN
1,1,24,105574,2013-01-03,59692.0,1.0,False,2013,1,3,3,NaN
2,2,24,105574,2013-01-04,99664.0,3.0,False,2013,1,4,4,NaN
3,3,24,105574,2013-01-05,140805.0,4.0,False,2013,1,5,5,NaN
4,4,24,105574,2013-01-06,182800.0,7.0,False,2013,1,6,6,NaN


In [57]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35069356 entries, 0 to 35069355
Data columns (total 12 columns):
 #   Column             Dtype         
---  ------             -----         
 0   Unnamed: 0         int64         
 1   store_nbr          int64         
 2   item_nbr           int64         
 3   date               datetime64[ns]
 4   id                 float64       
 5   unit_sales         float64       
 6   onpromotion        object        
 7   year               int64         
 8   month              int64         
 9   day                int64         
 10  day_of_week        int64         
 11  unit_sales_7d_avg  float64       
dtypes: datetime64[ns](1), float64(3), int64(7), object(1)
memory usage: 3.1+ GB


In [58]:
train_df['onpromotion'] = train_df['onpromotion'].fillna(False).astype(bool)
train_df['onpromotion'] = train_df['onpromotion'].astype('int64')


In [59]:
# drop column Unnamed 
train_df = train_df.drop(columns=['Unnamed: 0'])

In [60]:
# Filter to Jan-Mar 2014 only
train_df = train_df[(train_df['date'] >= '2014-01-01') & (train_df['date'] < '2014-04-01')]


In [61]:
train_df.shape

(1870920, 11)

In [62]:
train_df.isnull().sum()

store_nbr                  0
item_nbr                   0
date                       0
id                   1241775
unit_sales                 0
onpromotion                0
year                       0
month                      0
day                        0
day_of_week                0
unit_sales_7d_avg          0
dtype: int64

In [63]:
# Drop 'id' column
# 'id' column is not needed for the analysis
train_df.drop(columns=['id'], inplace=True)

In [64]:
train_df.isnull().sum()

store_nbr            0
item_nbr             0
date                 0
unit_sales           0
onpromotion          0
year                 0
month                0
day                  0
day_of_week          0
unit_sales_7d_avg    0
dtype: int64

# Merging External Datasets

In [65]:
# Load and merge store info
stores = pd.read_csv("data/stores.csv")
train_df = train_df.merge(stores, on="store_nbr", how="left")


In [66]:
# Load and merge item info
items = pd.read_csv("data/items.csv")
train_df = train_df.merge(items, on="item_nbr", how="left")

In [67]:
# Merge oil prices
oil = pd.read_csv("data/oil.csv")
oil['date'] = pd.to_datetime(oil['date'])
oil.rename(columns={'dcoilwtico': 'oil_price'}, inplace=True)
oil['oil_price'].fillna(method='ffill', inplace=True)
train_df = train_df.merge(oil, on='date', how='left')

In [68]:
# Merge transactions
trans = pd.read_csv("data/transactions.csv")
trans['date'] = pd.to_datetime(trans['date'])
train_df = train_df.merge(trans, on=['date', 'store_nbr'], how='left')
train_df['transactions'].fillna(0, inplace=True)


In [69]:
# Merge holidays
holidays = pd.read_csv("data/holidays_events.csv")
holidays['date'] = pd.to_datetime(holidays['date'])
holidays = holidays[['date', 'type']]
holidays['is_holiday'] = 1
train_df = train_df.merge(holidays, on='date', how='left')
train_df['is_holiday'] = train_df['is_holiday'].fillna(0)


In [70]:
# 🎉 One-Hot Encode Holiday Type ('type_y')
#holiday_dummies = pd.get_dummies(train_df['type_y'], prefix='holiday_type')
#train_df = pd.concat([train_df, holiday_dummies], axis=1)
#train_df.drop(columns=['type_y'], inplace=True)

# Feature Engineering

In [71]:
train_df['day_of_week'] = train_df['date'].dt.dayofweek
train_df['dow_sin'] = np.sin(2 * np.pi * train_df['day_of_week'] / 7)
train_df['dow_cos'] = np.cos(2 * np.pi * train_df['day_of_week'] / 7)
train_df.sort_values(by=['store_nbr', 'item_nbr', 'date'], inplace=True)
group = train_df.groupby(['store_nbr', 'item_nbr'])
train_df['lag_1'] = group['unit_sales'].shift(1)
train_df['lag_7'] = group['unit_sales'].shift(7)
train_df['rolling_mean_7'] = group['unit_sales'].shift(1).rolling(7).mean()
train_df['rolling_std_7'] = group['unit_sales'].shift(1).rolling(7).std()
train_df = train_df.dropna(subset=['lag_1', 'lag_7', 'rolling_mean_7', 'rolling_std_7'])

In [72]:
train_df.head()

,store_nbr,item_nbr,date,unit_sales,onpromotion,year,month,day,day_of_week,unit_sales_7d_avg,...,oil_price,transactions,type_y,is_holiday,dow_sin,dow_cos,lag_1,lag_7,rolling_mean_7,rolling_std_7
609307,24,96995,2014-01-08,0.0,0,2014,1,8,2,0.857143,...,91.90,2465.0,NaN,0.0,0.974928,-0.222521,1.0,0.0,0.857143,1.214986
609308,24,96995,2014-01-09,1.0,0,2014,1,9,3,0.571429,...,91.36,2176.0,NaN,0.0,0.433884,-0.900969,0.0,3.0,0.857143,1.214986
609309,24,96995,2014-01-10,0.0,0,2014,1,10,4,0.285714,...,92.39,2367.0,NaN,0.0,-0.433884,-0.900969,1.0,2.0,0.571429,0.786796
609310,24,96995,2014-01-11,0.0,0,2014,1,11,5,0.285714,...,NaN,2344.0,NaN,0.0,-0.974928,-0.222521,0.0,0.0,0.285714,0.487950
609311,24,96995,2014-01-12,1.0,0,2014,1,12,6,0.428571,...,NaN,1715.0,NaN,0.0,-0.781831,0.623490,0.0,0.0,0.285714,0.487950


In [73]:
train_df.isnull().sum()

store_nbr                  0
item_nbr                   0
date                       0
unit_sales                 0
onpromotion                0
year                       0
month                      0
day                        0
day_of_week                0
unit_sales_7d_avg          0
city                       0
state                      0
type_x                     0
cluster                    0
family                     0
class                      0
perishable                 0
oil_price             498912
transactions               0
type_y               1663040
is_holiday                 0
dow_sin                    0
dow_cos                    0
lag_1                      0
lag_7                      0
rolling_mean_7             0
rolling_std_7              0
dtype: int64

In [74]:
train_df['oil_price'].fillna(method='ffill', inplace=True)


In [75]:
train_df.drop(columns=['type_y'], inplace=True)
